In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("MovieLens")

from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc= spark.sparkContext

22/03/27 23:03:39 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/27 23:03:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e867aa0f-6964-4bed-a634-4091ba5c7330;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 542ms :: artifacts dl 9ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central 

In [3]:
from pyspark.sql.types import StructType, LongType, StringType, IntegerType, DoubleType

movieSchema = StructType()\
                    .add('movieId', IntegerType(), True)\
                    .add('title', StringType(), True)\
                    .add('genres', StringType(), True)

ratingSchema =  StructType()\
                    .add('userId', IntegerType(), True)\
                    .add('movieId', StringType(), True)\
                    .add('rating', StringType(), True)\
                    .add('timestamp', LongType(), True)

In [4]:
movieDf = spark.read.format('csv')\
                .option('header', True)\
                .schema(movieSchema)\
                .load('hdfs://localhost:9000/movies')

movieDf.printSchema()
movieDf.show(2)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



In [5]:
ratingDf = spark.read.format('csv')\
                .option('header', True)\
                .schema(ratingSchema)\
                .load('hdfs://localhost:9000/ratings')

ratingDf.printSchema()
ratingDf.show(2)

root
 |-- userId: integer (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



In [6]:
print (movieDf.count())
print (ratingDf.count())

9742
100836


In [7]:
ratingDf.take(2)

[Row(userId=1, movieId='1', rating='4.0', timestamp=964982703),
 Row(userId=1, movieId='3', rating='4.0', timestamp=964981247)]

In [8]:
ratingDf.select('rating').distinct().show()

+------+
|rating|
+------+
|   1.0|
|   4.5|
|   2.5|
|   3.5|
|   5.0|
|   0.5|
|   4.0|
|   1.5|
|   2.0|
|   3.0|
+------+



In [9]:
from pyspark.sql.functions import col, desc, avg, count

df = ratingDf\
    .groupBy('movieId')\
    .agg(count('userId').alias('total_ratings'))


df.printSchema()
df.show(20)

root
 |-- movieId: string (nullable = true)
 |-- total_ratings: long (nullable = false)

+-------+-------------+
|movieId|total_ratings|
+-------+-------------+
|    296|          307|
|   1090|           63|
| 115713|           28|
|   3210|           42|
|  88140|           32|
|    829|            9|
|   2088|           18|
|   2294|           45|
|   4821|            5|
|  48738|           20|
|   3959|            8|
|  89864|           19|
|   2136|           14|
|    691|            3|
|   3606|            4|
| 121007|            1|
|   6731|            8|
|  27317|            6|
|  26082|            3|
| 100553|            2|
+-------+-------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import col, desc, avg, count

df = ratingDf\
    .groupBy('movieId')\
    .agg(avg('rating').alias('avg_rating'))\
    .sort(desc('avg_rating'))


df.printSchema()
df.show(20)

root
 |-- movieId: string (nullable = true)
 |-- avg_rating: double (nullable = true)



+-------+----------+
|movieId|avg_rating|
+-------+----------+
| 179135|       5.0|
| 151769|       5.0|
|  69469|       5.0|
| 102217|       5.0|
|    495|       5.0|
|  47736|       5.0|
| 149350|       5.0|
| 170597|       5.0|
|   5059|       5.0|
| 136447|       5.0|
| 149508|       5.0|
| 136355|       5.0|
| 140627|       5.0|
| 147330|       5.0|
| 139640|       5.0|
|    467|       5.0|
|   6402|       5.0|
|   1349|       5.0|
|  67618|       5.0|
| 160644|       5.0|
+-------+----------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import col, desc, avg, count

mostPopularMoviesDf = ratingDf\
    .groupBy('movieId')\
    .agg(avg('rating').alias('avg_rating'), count('userId').alias('total_ratings'))\
    .filter ((col('total_ratings') >= 100 ) & (col('avg_rating') >=3.5))\
    .sort(desc('total_ratings'))

mostPopularMoviesDf.cache()

mostPopularMoviesDf.printSchema()
mostPopularMoviesDf.show(20)

root
 |-- movieId: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- total_ratings: long (nullable = false)



+-------+------------------+-------------+
|movieId|        avg_rating|total_ratings|
+-------+------------------+-------------+
|    356| 4.164133738601824|          329|
|    318| 4.429022082018927|          317|
|    296| 4.197068403908795|          307|
|    593| 4.161290322580645|          279|
|   2571| 4.192446043165468|          278|
|    260| 4.231075697211155|          251|
|    480|              3.75|          238|
|    110| 4.031645569620253|          237|
|    589| 3.970982142857143|          224|
|    527|             4.225|          220|
|   2959| 4.272935779816514|          218|
|      1|3.9209302325581397|          215|
|   1196|4.2156398104265405|          211|
|   2858| 4.056372549019608|          204|
|     50| 4.237745098039215|          204|
|     47|3.9753694581280787|          203|
|    150| 3.845771144278607|          201|
|   1198|            4.2075|          200|
|   4993| 4.106060606060606|          198|
|   1210| 4.137755102040816|          196|
+-------+--

In [12]:
popularMoviesDf = mostPopularMoviesDf.join(movieDf, mostPopularMoviesDf.movieId == movieDf.movieId)\
                                    .select(movieDf.movieId, 'title', 'avg_rating', 'total_ratings')\
                                    .sort (desc ('total_ratings'))

popularMoviesDf.show(100)

+-------+--------------------+------------------+-------------+
|movieId|               title|        avg_rating|total_ratings|
+-------+--------------------+------------------+-------------+
|    356| Forrest Gump (1994)| 4.164133738601824|          329|
|    318|Shawshank Redempt...| 4.429022082018927|          317|
|    296| Pulp Fiction (1994)| 4.197068403908795|          307|
|    593|Silence of the La...| 4.161290322580645|          279|
|   2571|  Matrix, The (1999)| 4.192446043165468|          278|
|    260|Star Wars: Episod...| 4.231075697211155|          251|
|    480|Jurassic Park (1993)|              3.75|          238|
|    110|   Braveheart (1995)| 4.031645569620253|          237|
|    589|Terminator 2: Jud...| 3.970982142857143|          224|
|    527|Schindler's List ...|             4.225|          220|
|   2959|   Fight Club (1999)| 4.272935779816514|          218|
|      1|    Toy Story (1995)|3.9209302325581397|          215|
|   1196|Star Wars: Episod...|4.21563981

In [13]:
popularMoviesDf.write.mode('overwrite')\
                .option('header', True)\
                .csv('hdfs://localhost:9000/most-popular-movies-many-files')

In [14]:
popularMoviesDf.withColumnRenamed("Company Name", "Company")\
        .withColumnRenamed("ISIN Code", "ISIN")\
        .write\
        .mode("overwrite")\
        .format("jdbc")\
        .option("url", "jdbc:mysql://localhost:3306/moviedb")\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("user", "team")\
        .option("password", "Team1234!")\
        .option("dbtable", "popular_movies")\
        .save()

Sun Mar 27 23:04:09 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Sun Mar 27 23:04:11 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

In [15]:
MovieDataFromMySqlDf =( spark.read
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/moviedb")
.option("driver", "com.mysql.jdbc.Driver")
.option("user", "team")
.option("password", "Team1234!")
.option("dbtable", "popular_movies")
.load()
                       )

MovieDataFromMySqlDf.printSchema()
MovieDataFromMySqlDf.show(5)

Sun Mar 27 23:04:13 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Sun Mar 27 23:04:13 IST 2022 WARN: Caught while disconnecting...

EXCEPTION STACK TRACE:



** BEGIN NESTED EXCEPTION ** 

javax.net.ssl.SSLException
MESSAGE: closing inbound before receiving peer's close_notify

STACKTRACE:

javax.net.ssl.SSLException: closing inbound before receiving peer's close_notify
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:735)
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:714)
	at com.mysql.jdbc.MysqlIO.quit

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- total_ratings: long (nullable = true)

+-------+--------------------+-----------------+-------------+
|movieId|               title|       avg_rating|total_ratings|
+-------+--------------------+-----------------+-------------+
|    356| Forrest Gump (1994)|4.164133738601824|          329|
|    318|Shawshank Redempt...|4.429022082018927|          317|
|    296| Pulp Fiction (1994)|4.197068403908795|          307|
|    593|Silence of the La...|4.161290322580645|          279|
|   2571|  Matrix, The (1999)|4.192446043165468|          278|
+-------+--------------------+-----------------+-------------+
only showing top 5 rows



Sun Mar 27 23:04:13 IST 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Sun Mar 27 23:04:13 IST 2022 WARN: Caught while disconnecting...

EXCEPTION STACK TRACE:



** BEGIN NESTED EXCEPTION ** 

javax.net.ssl.SSLException
MESSAGE: closing inbound before receiving peer's close_notify

STACKTRACE:

javax.net.ssl.SSLException: closing inbound before receiving peer's close_notify
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:735)
	at sun.security.ssl.SSLSocketImpl.shutdownInput(SSLSocketImpl.java:714)
	at com.mysql.jdbc.MysqlIO.quit